# Scenario Report (For Public Use)

In [ ]:
# Copyright (c) 2021 Gregory J. Miller. All rights reserved.
# Licensed under the Apache License, Version 2.0, which is in the LICENSE file.

from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly
import plotly.graph_objects as go 
import numpy as np

#get the name of the current directory to specify the scenario name and identify the output directory
scenario_name = str(Path.cwd()).split('\\')[-1]
if scenario_name == 'inputs':
    data_dir = Path.cwd() / '../outputs/'
    inputs_dir = Path.cwd() / '../inputs/'
    scenario_name = 'N/A'
else:
    data_dir = Path.cwd() / f'../../outputs/{scenario_name}/'
    inputs_dir = Path.cwd() / f'../../inputs/{scenario_name}/'

#define formatting options/functions for outputs
pd.options.display.float_format = '{:,.2f}'.format
def format_currency(x): 
    try:
        formatted = '$ {:,.2f}'.format(x)
    except ValueError:
        formatted = x
    return formatted
def format_percent(x): 
    try:
        formatted = '{:,.2f}%'.format(x)
    except ValueError:
        formatted = x
    return formatted

#allow the notebook to display plots in html report
###################################################
plotly.offline.init_notebook_mode()

print(f'Scenario Name: {scenario_name}')

## Portfolio Renewable Percentage

In [ ]:
# Calculate Time-coincident renewable % and annual renewable %
##############################################################

# load load balance data    
tcr = pd.read_csv(data_dir / 'load_balance.csv', parse_dates=True)

#calculate the time-coincident renewable percentage
try:
    tc_percent_renewable = (tcr['ZoneTotalGeneratorDispatch'].sum() + tcr['ZoneTotalStorageDispatch'].sum()) / tcr['zone_demand_mw'].sum() * 100
except KeyError:
    tc_percent_renewable = (tcr['ZoneTotalGeneratorDispatch'].sum()) / tcr['zone_demand_mw'].sum() * 100

# load excess generation data
excess = pd.read_csv(data_dir / 'dispatch.csv', parse_dates=True, usecols=['timestamp','Excess_Gen_MW'])
excess = excess.groupby('timestamp').sum().reset_index()

#merge excess gen data in 
tcr = tcr.merge(excess, how='left', on='timestamp')
tcr['Total_Gen_MW'] = tcr['ZoneTotalGeneratorDispatch'] + tcr['Excess_Gen_MW']
tcr['Time_Coincident_Generation'] = tcr[['Total_Gen_MW','zone_demand_mw']].min(axis=1)

tc_no_storage_percent_renewable = tcr['Time_Coincident_Generation'].sum() / tcr['zone_demand_mw'].sum() * 100
annual_percent_renewable = tcr['Total_Gen_MW'].sum() / tcr['zone_demand_mw'].sum() * 100

print(f'Time-coincident renewable percentage (with storage dispatch): {np.round(tc_percent_renewable, decimals=2)}%')
print(f'Time-coincident renewable percentage (without storage dispatch): {np.round(tc_no_storage_percent_renewable, decimals=2)}%')
print(f'Annual volumetric renewable percentage: {annual_percent_renewable.round(decimals=2)}%')

## Generator Portfolio
The sunburst chart describes the built portfolio at various levels of detail, which shows how the outer rings relate to the inner rings
- Inner circle: contract status (contracted or additional project)
- Outer ring: technology type (e.g. solar, wind, ...)


In [ ]:
# Prepare data for sunburst chart of portfolio
##############################################

#load capacity costs
portfolio = pd.read_csv(data_dir / 'gen_cap.csv', usecols=['generation_project','GenCapacity'])
portfolio = portfolio[portfolio['GenCapacity'] > 0]
portfolio = portfolio.rename(columns={'GenCapacity':'MW'})

# load data about predetermined generators
existing = pd.read_csv(inputs_dir / 'gen_build_predetermined.csv', usecols=['GENERATION_PROJECT']).rename(columns={'GENERATION_PROJECT':'generation_project'})


# add column indicating which generators are contracted or additional builds

existing['Status'] = 'Contracted'
portfolio = portfolio.merge(existing, how='left', on='generation_project').fillna('Additional')


#split the prefix of the generator name from the rest of the string
tech_column = [i.split('_')[0] for i in portfolio['generation_project']]
generator_name = [' '.join(i.split('_')[1:]) for i in portfolio['generation_project']]

#create a new tecnhology column based on the prefix and drop the generator name
portfolio['Technology'] = tech_column
portfolio['generation_project'] = generator_name

portfolio['MW'] = portfolio['MW'].round(0)

portfolio = portfolio.sort_values(by=['Status','Technology'])


portfolio_sunburst = px.sunburst(portfolio, path=['Status','Technology'], values='MW', color='Technology', color_discrete_map={'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'Yellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'Green',
 '(?)':'Black'},
 width=1000, height=1000,
 title='Energy Portfolio by Technology Type and Contract Status (MW)')
portfolio_sunburst.update_traces(textinfo='label+value')
portfolio_sunburst.show()

## Delivered Energy Mix
This is similar to a power content label, showing the percentage of energy from each source used to serve load.

In [ ]:
energy_mix = pd.read_csv(data_dir / 'generation.csv', usecols=['generation_project','Energy_GWh_typical_yr'])
energy_mix= energy_mix.round(decimals=2)
energy_mix = energy_mix[energy_mix['Energy_GWh_typical_yr'] > 0]
energy_mix['generation_project'] = [i.split('_')[0] for i in energy_mix['generation_project']]

try:
    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=True, usecols=['generation_project','timestamp','DispatchMW'])
    storage = storage.groupby('generation_project').sum().reset_index()
    
    #rename storage hybrid to paired generator name
    hybrid_pair = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','storage_hybrid_generation_project'])
    hybrid_pair = hybrid_pair[hybrid_pair['storage_hybrid_generation_project'] != "."]
    hybrid_pair = dict(zip(hybrid_pair.GENERATION_PROJECT, hybrid_pair.storage_hybrid_generation_project))
    storage['generation_project'] = storage['generation_project'].replace(hybrid_pair)

    storage['generation_project'] = [i.split('_')[0] for i in storage['generation_project']]
    storage = storage.rename(columns={'DispatchMW':'Energy_GWh_typical_yr'})
    #convert MWh to GWh
    storage['Energy_GWh_typical_yr'] = storage['Energy_GWh_typical_yr'] / 1000

    #append the storage data to the energy mix data, and groupby generator name to combine hybrid components
    energy_mix = energy_mix.append(storage)
    

except FileNotFoundError:
    pass

#append system power data
try:
    system_power = pd.read_csv(data_dir / 'system_power.csv')[['System_Power_GWh_per_year']].sum()
    energy_mix = energy_mix.append(pd.DataFrame({'generation_project':['SYSTEM_POWER'], 'Energy_GWh_typical_yr':[system_power]}))
except FileNotFoundError:
    pass

energy_mix = energy_mix.groupby('generation_project').sum().reset_index()
energy_mix = energy_mix.rename(columns={'generation_project':'Technology','Energy_GWh_typical_yr':'Delivered Energy GWh'})
energy_mix['Delivered Energy GWh'] = energy_mix['Delivered Energy GWh'].round(decimals=0)


delivered_energy_pie = px.pie(energy_mix, values='Delivered Energy GWh', names='Technology', title='Delivered Energy Mix (GWh)', color='Technology', color_discrete_map={'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'Yellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'Green',
 'SYSTEM_POWER':'Red',
 'Total':'Black'},
width=600, height=600
 )
delivered_energy_pie.update_traces(textinfo='percent+label+value')
delivered_energy_pie.show()

## Delivered Electricity Cost Summary

In [ ]:
try:
    # Determine excess costs

    excess_costs = pd.read_csv(data_dir / 'congestion_costs_by_gen.csv', parse_dates=['timestamp'], infer_datetime_format=True)

    #remove rows that are all zero
    excess_costs = excess_costs[~((excess_costs['DispatchGen_MW'].round(decimals=2) + excess_costs['Excess_Gen_MW'].round(decimals=2)) == 0)]

    excess_costs['PPA_Cost_Per_MWh'] = excess_costs['Contract Cost'] / (excess_costs['DispatchGen_MW'] + excess_costs['Excess_Gen_MW'])

    excess_costs['Pnode_Cost_Per_MWh'] = (excess_costs['Dispatched Pnode Revenue'] + excess_costs['Excess Pnode Revenue']) / (excess_costs['DispatchGen_MW'] + excess_costs['Excess_Gen_MW'])

    #merge charging data for hybrid storage projects into the dataframe
    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    hybrid_charge = storage[['generation_project','timestamp','ChargeMW']]
    hybrid_charge['generation_project'] = hybrid_charge['generation_project'].replace(hybrid_pair)
    excess_costs = excess_costs.merge(hybrid_charge, how='left', on=['generation_project','timestamp']).fillna(0)

    #calculate the true excess of hybrid projects (any energy that was not charged)
    excess_costs['Actual_Excess_Gen_MW'] = excess_costs['Excess_Gen_MW'] - excess_costs['ChargeMW']
    excess_costs['Actual_Excess_Contract_Cost'] = excess_costs['Actual_Excess_Gen_MW'] * excess_costs['PPA_Cost_Per_MWh']
    excess_costs['Actual_Excess_Pnode_Revenue'] = excess_costs['Actual_Excess_Gen_MW'] * excess_costs['Pnode_Cost_Per_MWh']

    # round all values to two decimal points
    excess_costs = excess_costs.round(decimals=2)

    # now we want to aggregate the data by timestamp
    excess_costs_in_tp = excess_costs[['timestamp','Actual_Excess_Gen_MW','Actual_Excess_Contract_Cost','Actual_Excess_Pnode_Revenue']].groupby('timestamp').sum()

    #now we want to merge charging data for non-hybrid storage 
    storage_charge = storage[['generation_project','timestamp','ChargeMW']]
    storage_charge = storage_charge[~storage_charge['generation_project'].str.contains('HYBRID')]
    storage_charge = storage_charge.groupby('timestamp').sum()
    excess_costs_in_tp = excess_costs_in_tp.merge(storage_charge, how='left', left_index=True, right_index=True)

    excess_costs_in_tp = excess_costs_in_tp.round(decimals=2)
    excess_costs_in_tp = excess_costs_in_tp[~((excess_costs_in_tp['Actual_Excess_Gen_MW'].round(decimals=2) + excess_costs_in_tp['ChargeMW'].round(decimals=2)) == 0)]

    #calculate teh average contract and pnode cost per mwh of the excess
    excess_costs_in_tp['PPA_Cost_Per_MWh'] = excess_costs_in_tp['Actual_Excess_Contract_Cost'] / excess_costs_in_tp['Actual_Excess_Gen_MW']
    excess_costs_in_tp['Pnode_Cost_Per_MWh'] = excess_costs_in_tp['Actual_Excess_Pnode_Revenue'] / excess_costs_in_tp['Actual_Excess_Gen_MW']

    excess_costs_in_tp = excess_costs_in_tp.fillna(0)

    #calculate the cost/revenue for generator energy to storage
    excess_costs_in_tp['Energy_For_Charging_Contract_Cost'] = excess_costs_in_tp['ChargeMW'] * excess_costs_in_tp['PPA_Cost_Per_MWh']
    excess_costs_in_tp['Energy_For_Charging_Pnode_Revenue'] = excess_costs_in_tp['ChargeMW'] * excess_costs_in_tp['Pnode_Cost_Per_MWh']

    #now recalculate the actual excess gen, contract costs, and pnode revenues accounting for storage
    excess_costs_in_tp['Actual_Excess_Gen_MW'] = excess_costs_in_tp['Actual_Excess_Gen_MW'] - excess_costs_in_tp['ChargeMW']
    excess_costs_in_tp['Actual_Excess_Contract_Cost'] = excess_costs_in_tp['Actual_Excess_Gen_MW'] * excess_costs_in_tp['PPA_Cost_Per_MWh']
    excess_costs_in_tp['Actual_Excess_Pnode_Revenue'] = excess_costs_in_tp['Actual_Excess_Gen_MW'] * excess_costs_in_tp['Pnode_Cost_Per_MWh']

    # Summarize Portfolio Cost
    ##########################

    system_demand_mwh = pd.read_csv(data_dir / 'electricity_cost.csv')['SystemDemand_MWh'].to_numpy()[0]

    #load itemized cost data
    costs_itemized_df = pd.read_csv(data_dir / 'costs_itemized.csv')

    #calculate delivered costs

    # 1) Generation Energy Cost
    generation_cost = (costs_itemized_df[costs_itemized_df['Component'] == 'GenPPACostInTP']['AnnualCost_Real'].to_numpy()[0] + \
        excess_costs_in_tp['Energy_For_Charging_Contract_Cost'].sum()) / system_demand_mwh

    # 2) Excess generation energy cost
    excess_generation_cost = excess_costs_in_tp['Actual_Excess_Contract_Cost'].sum() / system_demand_mwh


    # 3) Storage capacity cost
    storage_capacity_cost = costs_itemized_df[costs_itemized_df['Component'] == 'TotalGenCapacityCost']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh

    """
    try:
        weighted_storage = costs_itemized_df[costs_itemized_df['Component'] == 'TotalGenCapacityCost']['AnnualCost_Real'].to_numpy()[0] / tech_portfolio[tech_portfolio['Technology'] == 'STORAGE']['MW'].to_numpy()[0] / 12000
    except IndexError:
        weighted_storage = 0
    """

    # 4) Storage Energy Arbitrage
    try:
        storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
        net_storage_cost = storage['StorageDispatchPnodeCost'].sum(axis=0)
        storage_arbitrage_cost = net_storage_cost / system_demand_mwh

    except FileNotFoundError:
        storage_arbitrage_cost = 0




    # 6) System Power Costs
    try:
        system_power_cost = costs_itemized_df[costs_itemized_df['Component'] == 'SystemPowerCost']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
        
    except:
        system_power_cost = 0

    # 5) Congestion Costs
    try:
        dlap_cost = costs_itemized_df[costs_itemized_df['Component'] == 'DLAPLoadCostInTP']['AnnualCost_Real'].to_numpy()[0] / system_demand_mwh
        pnode_revenue = (costs_itemized_df[costs_itemized_df['Component'] == 'DispatchedGenPnodeRevenueInTP']['AnnualCost_Real'].to_numpy()[0] + \
            costs_itemized_df[costs_itemized_df['Component'] == 'StorageDispatchPnodeCostInTP']['AnnualCost_Real'].to_numpy()[0] - \
                excess_costs_in_tp['Energy_For_Charging_Pnode_Revenue'].sum()) / system_demand_mwh
        excess_pnode = - excess_costs_in_tp['Actual_Excess_Pnode_Revenue'].sum() / system_demand_mwh
    except:
        dlap_cost = 0
        pnode_revenue = 0
        excess_pnode = 0


    cost = {
        'Cost Component': ['Generation','Excess Generation','Pnode Revenue','Excess Pnode Revenue','Storage Capacity','Storage Energy Arbitrage','DLAP Load Cost'],
        '$/MWh delivered': [generation_cost, excess_generation_cost, pnode_revenue, excess_pnode, storage_capacity_cost, storage_arbitrage_cost, dlap_cost],
        #'Weighted Cost': [weighted_generation, weighted_excess, weighted_storage, weighted_arbitrage, weighted_system],
        #'Unit': ['$/MWh','$/MWh','$/kW-mo', '$/MWh', '$/MWh']
    }

    cost_summary = pd.DataFrame(cost)
    cost_summary = cost_summary.round(decimals=2)

    # Calculate RA Costs
    #######################################

    try:

        ra_open = pd.read_csv(data_dir / 'RA_open_position.csv')
        
        #where RA Position is negative, it indicates an open position
        ra_open['Color'] = np.where(ra_open["RA_Position_MW"]<0, 'Open Position', 'Excess RA')

        #create a plot of monthly RA position
        monthly_ra_open_fig = px.bar(ra_open, x='Month', y='RA_Position_MW', facet_col='RA_Requirement', color='Color', color_discrete_map={'Excess RA':'green', 'Open Position':'red'}, title='Monthly RA position by requirement')
        monthly_ra_open_fig.for_each_annotation(lambda a: a.update(text=a.text.replace("RA_Requirement=", "")))

        sellable_flex = ra_open[(ra_open['RA_Requirement'] == 'flexible_RA') & (ra_open['RA_Position_MW'] > 0)]

        #flex RA must be paired with regular RA to sell, so we need to limit it
        def calculate_sellable_flex(row, ra_open):
            # get the month number
            month = row['Month']
            #get the amount of excess system RA
            system_ra_open_MW = ra_open.loc[(ra_open['RA_Requirement'] == 'system_RA') & (ra_open['Month'] == month), 'Excess_RA_MW'].item()
            # find the minimum of the excess system RA and excess flex RA
            sellable = min(row['RA_Position_MW'], system_ra_open_MW)
            return sellable   
        sellable_flex['Sellable_Flex_MW'] = sellable_flex.apply(lambda row: calculate_sellable_flex(row, ra_open), axis=1)

        #re-calculate the sellable position of flex RA
        sellable_flex['Excess_RA_Value'] = sellable_flex['RA_Value'] * sellable_flex['Sellable_Flex_MW']

        #calculate how much system could be sold for subtracting the local
        sellable_system = ra_open[(ra_open['RA_Requirement'] == 'system_RA')]

        local_RA = ra_open[(ra_open['RA_Requirement'] != 'system_RA') & (ra_open['RA_Requirement'] != 'flexible_RA')][['Month','Excess_RA_MW']]
        local_RA = local_RA.groupby('Month').sum().reset_index().rename(columns={'Excess_RA_MW':'Local_RA_MW'})

        #merge local RA data into system data
        sellable_system = sellable_system.merge(local_RA, how='left', on='Month').fillna(0)
        sellable_system['Sellable_System_MW'] = sellable_system['Excess_RA_MW'] - sellable_system['Local_RA_MW']

        sellable_system['Excess_RA_Value'] = sellable_system['RA_Value'] * sellable_system['Sellable_System_MW']

        #calculate total RA costs and value
        total_RA_open_cost = ra_open['Open_Position_Cost'].sum()
        excess_flex_RA_value = sellable_flex['Excess_RA_Value'].sum()
        excess_local_RA_value = ra_open.loc[(ra_open['RA_Requirement'] != 'system_RA') & (ra_open['RA_Requirement'] != 'flexible_RA'), 'Excess_RA_Value'].sum()
        excess_system_RA_value = sellable_system['Excess_RA_Value'].sum()

        #calculate the best-case RA cost if all RA can be sold at full market value
        #ra_cost_best_case = (total_RA_open_cost - (excess_flex_RA_value + excess_local_RA_value + excess_system_RA_value)) / system_demand_mwh


        #create cost summary of RA costs
        ################################

        ra_costs = ra_open.copy()
        #groupby RA requirement
        ra_costs = ra_costs.groupby('RA_Requirement').mean().drop(columns=['Period','Month'])
        #rename columns
        ra_costs = ra_costs.rename(columns={'Open_Position_MW': 'Monthly_Avg_Open_Position_MW', 'Open_Position_Cost': 'Monthly_Avg_Open_Position_Cost','RA_Requirement_Need_MW':'Monthly_Avg_RA_Need_MW','Available_RA_Capacity_MW':'Monthly_Avg_Available_RA_MW'})

        
        #calculate the annual total
        ra_costs['Annual_Open_Position_Cost'] = ra_costs['Monthly_Avg_Open_Position_Cost'] * 12
        ra_costs['$/MWh delivered'] = ra_costs['Annual_Open_Position_Cost'] / system_demand_mwh
        #ra_costs['Weighted Cost'] = ra_costs['RA_Cost'] / 1000
        #ra_costs['Unit'] = '$/kW-mo'
        ra_costs = ra_costs[['$/MWh delivered']]
        #ra_costs = ra_costs.drop(columns=['Monthly_Avg_Open_Position_Cost','Monthly_Avg_RA_Need_MW','Monthly_Avg_Available_RA_MW','Monthly_Avg_Open_Position_MW','Annual_Open_Position_Cost','RA_Position_MW', 'Excess_RA_MW','RA_Cost','RA_Value','Excess_RA_Value'])

        #merge RA cost data into cost_summary
        ra_costs = ra_costs.reset_index().rename(columns={'RA_Requirement':'Cost Component'})
        cost_summary = pd.concat([cost_summary, ra_costs])

        #calculate the total energy cost for the portfolio
        energy_cost_total = cost_summary['$/MWh delivered'].sum(axis=0)
        cost_summary = cost_summary.append({'Cost Component': 'Total','$/MWh delivered': energy_cost_total}, ignore_index=True)

        #Add total costs
        cost_summary['Total Annual Cost'] = cost_summary['$/MWh delivered'] * system_demand_mwh

        # calculate excess to sell
        sell_excess = cost_summary[cost_summary['Cost Component'].str.contains('Excess')]['$/MWh delivered'].sum()

        #calculate best and worst case portfolio costs
        worst_case_portfolio_cost = energy_cost_total
        mid_case_portfolio_cost = energy_cost_total - sell_excess
        best_case_portfolio_cost = energy_cost_total - sell_excess - ((excess_flex_RA_value + excess_local_RA_value + excess_system_RA_value) / system_demand_mwh)

    except FileNotFoundError:
        energy_cost_total = cost_summary['$/MWh delivered'].sum(axis=0)
        NO_RA = "N/A"

        #calculate the total energy cost for the portfolio
        energy_cost_total = cost_summary['$/MWh delivered'].sum(axis=0)
        cost_summary = cost_summary.append({'Cost Component': 'Total','$/MWh delivered': energy_cost_total}, ignore_index=True)

        #Add total costs
        cost_summary['Total Annual Cost'] = cost_summary['$/MWh delivered'] * system_demand_mwh

        # calculate excess to sell
        sell_excess = cost_summary[cost_summary['Cost Component'].str.contains('Excess')]['$/MWh delivered'].sum()

        #calculate best and worst case portfolio costs
        worst_case_portfolio_cost = energy_cost_total
        mid_case_portfolio_cost = energy_cost_total - sell_excess
        best_case_portfolio_cost = energy_cost_total - sell_excess

    print(f'The worst-case total portfolio cost per delivered MWh (if excess generation and excess RA cannot be sold) is  ${worst_case_portfolio_cost.round(decimals=2)} \n')
    print(f'The mid-case total portfolio cost per delivered MWh   (if all excess generation can be sold) is               ${mid_case_portfolio_cost.round(decimals=2)} \n')
    print(f'The best-case total portfolio cost per delivered MWh  (if all excess generation AND excess RA can be sold) is ${best_case_portfolio_cost.round(decimals=2)} \n')
except FileNotFoundError:
    pass

## Resource Adequacy Open Position

In [ ]:
try:
    monthly_ra_open_fig.show()
except:
    print('Resource Adequacy was not considered in this scenario')

# Supply and Demand Balance

In [ ]:
# Set up data for dispatch timeseries graph
###########################################

dispatch = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','generation_project','DispatchGen_MW', 'Excess_Gen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

#create a dictionary that matches hybrid storage to generator
hybrid_pair = pd.read_csv('generation_projects_info.csv', usecols=['GENERATION_PROJECT','storage_hybrid_generation_project'])
hybrid_pair = hybrid_pair[hybrid_pair['storage_hybrid_generation_project'] != "."]
hybrid_pair = dict(zip(hybrid_pair.GENERATION_PROJECT, hybrid_pair.storage_hybrid_generation_project))

try:
    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)

    #merge charging data for hybrid storage projects into the dataframe
    hybrid_dispatch = storage[['generation_project','timestamp','ChargeMW','DispatchMW']]
    hybrid_dispatch['generation_project'] = hybrid_dispatch['generation_project'].replace(hybrid_pair)
    dispatch = dispatch.merge(hybrid_dispatch, how='left', on=['generation_project','timestamp']).fillna(0)

    # subtract out charging energy from excess generation from hybrid projects to get true generation
    dispatch['Excess_Gen_MW'] = dispatch['Excess_Gen_MW'] - dispatch['ChargeMW']
    # add hybrid battery dispatch to DispatchGen_MW for hybrid projects to get true dispatch
    dispatch['DispatchGen_MW'] = dispatch['DispatchGen_MW'] + dispatch['DispatchMW']
    #drop the hybrid columns
    dispatch = dispatch.drop(columns=['ChargeMW','DispatchMW'])

    #calculate total generation
    dispatch['DispatchMW'] = dispatch['DispatchGen_MW'] + dispatch['Excess_Gen_MW']
    dispatch = dispatch.drop(columns=['DispatchGen_MW','Excess_Gen_MW'])

    #append storage 
    storage_dispatch = storage[['generation_project','timestamp','DispatchMW']]
    storage_dispatch = storage_dispatch[~storage_dispatch['generation_project'].str.contains('HYBRID')]
    dispatch = dispatch.append(storage_dispatch)

    # Setup storage charging timeseries data
    ########################################

    storage_charge = storage[['generation_project','timestamp','ChargeMW']]
    storage_charge = storage_charge[~storage_charge['generation_project'].str.contains('HYBRID')]
    storage_charge = storage_charge.groupby('timestamp').sum().reset_index()

except FileNotFoundError:
    #calculate total generation
    dispatch['DispatchMW'] = dispatch['DispatchGen_MW'] + dispatch['Excess_Gen_MW']
    dispatch = dispatch.drop(columns=['DispatchGen_MW','Excess_Gen_MW'])

#add system power data if available
try:
    system_power_use = pd.read_csv(data_dir / 'system_power.csv', usecols=['timestamp','System_Power_MW'], parse_dates=['timestamp'], infer_datetime_format=True)
    system_power_use['generation_project'] = 'SYSTEM_POWER'
    system_power_use = system_power_use.rename(columns={'System_Power_MW':'DispatchMW'})
    dispatch = dispatch.append(system_power_use)

except FileNotFoundError:
    pass

#add technology column
dispatch['Technology'] = [i.split('_')[0] for i in dispatch['generation_project']]

# group the data by technology type
dispatch_by_tech = dispatch.groupby(['Technology','timestamp']).sum().reset_index()
#only keep observations greater than 0
dispatch_by_tech = dispatch_by_tech[dispatch_by_tech['DispatchMW'] > 0]

# Setup load timeseries data
############################

load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

# Create Figure
###############

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'YellowGreen',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

dispatch_fig = px.area(dispatch_by_tech, x='timestamp', y='DispatchMW', color='Technology', color_discrete_map=color_map)
dispatch_fig.update_traces(line={'width':0})
#dispatch_fig.update_traces(hoveron="points+fills")
dispatch_fig.layout.template = 'plotly_white'
#hoveron = 'points+fills'

try:
    # add charging line
    dispatch_fig.add_scatter(x=storage_charge.timestamp, y=(storage_charge.ChargeMW + load.zone_demand_mw), line=dict(color='green', width=4), name='Load + Storage Charge')
except NameError:
    pass

# add load line
dispatch_fig.add_scatter(x=load.timestamp, y=load.zone_demand_mw, line=dict(color='black', width=4), name='Demand')

dispatch_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

dispatch_fig.show()

In [ ]:
# Battery State of Charge data
##############################

try:

    storage = pd.read_csv(data_dir / 'storage_dispatch.csv', parse_dates=['timestamp'], infer_datetime_format=True)
    
    soc = storage.pivot(index='timestamp', columns='generation_project', values='StateOfCharge')
    #remove columns where all values are 0
    soc = soc.loc[:, (soc != 0).any(axis=0)]
    
    #load storage capacity
    storage_energy_capacity = pd.read_csv(data_dir / 'storage_builds.csv', usecols=['generation_project','OnlineEnergyCapacityMWh'], index_col='generation_project')

    #create another dictionary of storage energy capacity summed by storage type
    storage_energy_capacity = storage_energy_capacity.reset_index()
    storage_energy_capacity['generation_project'] = [i.split('_')[0] for i in storage_energy_capacity['generation_project']]
    storage_energy_capacity = storage_energy_capacity.groupby('generation_project').sum()
    grouped_storage_energy_capacity_dict = storage_energy_capacity.to_dict()['OnlineEnergyCapacityMWh']


    #sum by storage type
    soc.columns = [i.split('_')[0] for i in soc.columns]
    
    soc = soc.groupby(soc.columns, axis=1).sum()
    
    #divide by the total capacity to get state of charge
    soc = soc.div(soc.assign(**grouped_storage_energy_capacity_dict))
    
    #soc.index = pd.to_datetime(soc.index)
    
    soc.head(5)

    soc_fig = px.line(soc, x=soc.index, y=list(soc.columns), color_discrete_map={'STORAGE':'green','STORAGEHYBRID':'yellowgreen'}, labels={'timestamp':'Datetime','value':'%'}, title='Storage State of Charge')

    soc_fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1d", step="day", stepmode="backward"),
            dict(count=7, label="1w", step="day", stepmode="backward"),
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(step="all")
        ])))

    soc_fig.show()
    
except FileNotFoundError:
    pass

In [ ]:
mh_dispatch = dispatch.copy()
mh_dispatch = mh_dispatch.set_index('timestamp')

#add a technology column
mh_dispatch['Technology'] = mh_dispatch['generation_project'].str.split('_', expand=True)[0]

#sum by technology
mh_dispatch = mh_dispatch.groupby(['Technology', mh_dispatch.index], axis=0).sum().reset_index(level=0)

#groupby month and hour
mh_dispatch = mh_dispatch.groupby(['Technology', mh_dispatch.index.month, mh_dispatch.index.hour], axis=0).mean()
mh_dispatch.index = mh_dispatch.index.rename(['Technology','Month','Hour'])
mh_dispatch = mh_dispatch.reset_index()


mh_dispatch = mh_dispatch[mh_dispatch['DispatchMW'] > 0]


#load data
mh_load = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], index_col='timestamp', parse_dates=True, infer_datetime_format=True).rename(columns={'zone_demand_mw':'DEMAND'})
mh_load = mh_load.groupby([mh_load.index.month, mh_load.index.hour], axis=0).mean()
mh_load.index = mh_load.index.rename(['Month','Hour'])
mh_load = mh_load.reset_index()

try:
    mh_charge = storage_charge.copy().set_index('timestamp')
    mh_charge = mh_charge.groupby([mh_charge.index.month, mh_charge.index.hour], axis=0).mean()
    mh_charge.index = mh_charge.index.rename(['Month','Hour'])
    mh_charge = mh_charge.reset_index()
    #merge load data
    mh_charge = mh_charge.merge(mh_load, how='left', on=['Month','Hour'])
    mh_charge['ChargeMW'] = mh_charge['ChargeMW'] + mh_charge['DEMAND']
except NameError:
    pass


# Generate the Figure
#####################

color_map = {'HYDRO':'Purple',
 'ONWIND':'Blue',
 'OFFWIND':'Navy',
 'PV':'Yellow',
 'PVHYBRID':'GreenYellow',
 'CSP':'Orange',
 'GEO':'Sienna',
 'STORAGE':'Green',
 'STORAGEHYBRID':'GreenYellow',
 'SYSTEM':'Red'}

mh_fig = px.area(mh_dispatch, x='Hour', y='DispatchMW', facet_col='Month', color='Technology', color_discrete_map=color_map, facet_col_wrap=6, width=1000, height=600, title='Month-Hour Generation Profiles')
mh_fig.layout.template = 'plotly_white'
#mh_fig.update_traces(line={'dash':'dot'})

try:
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 1, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 1, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=1, name='STORAGE_Charge', showlegend=True)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 2, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 2, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 3, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 3, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 4, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 4, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 5, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 5, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 6, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 6, 'ChargeMW'], line=dict(color='green', width=4), row=2, col=6, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 7, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 7, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=1, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 8, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 8, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=2, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 9, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 9, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=3, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 10, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 10, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=4, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 11, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 11, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=5, name='STORAGE_Charge', showlegend=False)
    mh_fig.add_scatter(x=mh_charge.loc[mh_charge['Month'] == 12, 'Hour'], y=mh_charge.loc[mh_charge['Month'] == 12, 'ChargeMW'], line=dict(color='green', width=4), row=1, col=6, name='STORAGE_Charge', showlegend=False)
except (NameError, KeyError) as e:
    pass

mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 1, 'Hour'], y=mh_load.loc[mh_load['Month'] == 1, 'DEMAND'], line=dict(color='black', width=4), row=2, col=1, name='Demand', showlegend=True)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 2, 'Hour'], y=mh_load.loc[mh_load['Month'] == 2, 'DEMAND'], line=dict(color='black', width=4), row=2, col=2, name='Demand',showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 3, 'Hour'], y=mh_load.loc[mh_load['Month'] == 3, 'DEMAND'], line=dict(color='black', width=4), row=2, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 4, 'Hour'], y=mh_load.loc[mh_load['Month'] == 4, 'DEMAND'], line=dict(color='black', width=4), row=2, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 5, 'Hour'], y=mh_load.loc[mh_load['Month'] == 5, 'DEMAND'], line=dict(color='black', width=4), row=2, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 6, 'Hour'], y=mh_load.loc[mh_load['Month'] == 6, 'DEMAND'], line=dict(color='black', width=4), row=2, col=6, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 7, 'Hour'], y=mh_load.loc[mh_load['Month'] == 7, 'DEMAND'], line=dict(color='black', width=4), row=1, col=1, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 8, 'Hour'], y=mh_load.loc[mh_load['Month'] == 8, 'DEMAND'], line=dict(color='black', width=4), row=1, col=2, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 9, 'Hour'], y=mh_load.loc[mh_load['Month'] == 9, 'DEMAND'], line=dict(color='black', width=4), row=1, col=3, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 10, 'Hour'], y=mh_load.loc[mh_load['Month'] == 10, 'DEMAND'], line=dict(color='black', width=4), row=1, col=4, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 11, 'Hour'], y=mh_load.loc[mh_load['Month'] == 11, 'DEMAND'], line=dict(color='black', width=4), row=1, col=5, name='Demand', showlegend=False)
mh_fig.add_scatter(x=mh_load.loc[mh_load['Month'] == 12, 'Hour'], y=mh_load.loc[mh_load['Month'] == 12, 'DEMAND'], line=dict(color='black', width=4), row=1, col=6, name='Demand', showlegend=False)

month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_fig.layout.annotations):
    a.text = month_names[i]


mh_fig.update_xaxes(dtick=3)
mh_fig.show()

In [ ]:
# get month-hour shape of open energy position
##############################################

dispatch_gen = pd.read_csv(data_dir / 'dispatch.csv', usecols=['timestamp','DispatchGen_MW', 'Excess_Gen_MW'], parse_dates=['timestamp'], infer_datetime_format=True)

#sum by timestamp
dispatch_gen = dispatch_gen.groupby('timestamp').sum().reset_index()

#sum to get total generation
dispatch_gen['Generation_MW'] = dispatch_gen['DispatchGen_MW'] + dispatch_gen['Excess_Gen_MW']

#merge generation_load_mismatch data
generation_load_mismatch = pd.read_csv(data_dir / 'load_balance.csv', usecols=['timestamp','zone_demand_mw'], parse_dates=['timestamp'], infer_datetime_format=True)

generation_load_mismatch = generation_load_mismatch.merge(dispatch_gen[['timestamp','Generation_MW']], how='left', on='timestamp')

#subtract generation from generation_load_mismatch to get open position and replace negative values with zero
generation_load_mismatch['Overgeneration_MW'] = generation_load_mismatch['Generation_MW'] - generation_load_mismatch['zone_demand_mw']
#generation_load_mismatch.loc[generation_load_mismatch['generation_load_mismatch_MW'] < 0, 'generation_load_mismatch_MW'] = 0

#generation_load_mismatch['timestamp'] = pd.to_datetime(generation_load_mismatch['timestamp'])

generation_load_mismatch = generation_load_mismatch.set_index('timestamp')

generation_load_mismatch_mh = generation_load_mismatch.groupby([generation_load_mismatch.index.month, generation_load_mismatch.index.hour], axis=0).mean()
generation_load_mismatch_mh.index = generation_load_mismatch_mh.index.rename(['Month','Hour'])
generation_load_mismatch_mh = generation_load_mismatch_mh.reset_index()

mh_mismatch_fig = px.area(generation_load_mismatch_mh, x='Hour', y='Overgeneration_MW', facet_col='Month', facet_col_wrap=6, width=1000, height=600, title='Shape of Over- and under-generation, excluding energy storage')
mh_mismatch_fig.update_xaxes(dtick=3)
month_names = ['July', 'August', 'September', 'October', 'November', 'December','January', 'February', 'March', 'April', 'May', 'June']
for i, a in enumerate(mh_mismatch_fig.layout.annotations):
    a.text = month_names[i]

mh_mismatch_fig.show()

